In [ ]:
import sys
import numpy as np
from scipy.stats import gaussian_kde
import theano.tensor as T
import theano
import os
import re
import glob
import h5py
from scipy.stats import norm
from keras import *
from sklearn.metrics import roc_curve, roc_auc_score, auc
from keras.models import Sequential,Model
from keras.layers import Dense, Activation,Input, Dense, Dropout, merge, Flatten
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Model
from keras.layers import Input, Dense, BatchNormalization
from keras.optimizers import *
from keras.models import model_from_json, save_model
from keras.layers import *
import matplotlib.pyplot as pt
%matplotlib inline
from keras import backend as K
from keras.engine import InputSpec, Layer
from keras import initializers, regularizers, constraints

In [ ]:
def loadmodel(name, weights = False):
    json_file = open('/home/kaustuv1993/Desktop/H2bb/Models_and_Weights/%s_m.json'%name, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    #load weights into new model
    if weights==True:
        model.load_weights('/home/kaustuv1993/Desktop/H2bb/Models_and_Weights/%s_w.h5'%name)
    print (model.summary())
    print("Loaded model %s from disk"%name)
    return model

def savemodel(model,name="neural network"):
    #print "Saving model:"
    model_name = name
    #model.summary()
    model.save_weights('/home/kaustuv1993/Desktop/H2bb/Models_and_Weights/%s_w.h5'%model_name, overwrite=True)
    model_json = model.to_json()
    with open("/home/kaustuv1993/Desktop/H2bb/Models_and_Weights/%s_m.json"%model_name, "w") as json_file:
        json_file.write(model_json)
        
def savelosses(hist, name="neural network"):  
    print "Saving losses:"
    loss = np.array(hist.history['loss'])
    valoss = np.array(hist.history['val_loss'])
    f = h5py.File("/home/kaustuv1993/Desktop/H2bb/Models_and_Weights/%s_h.h5"%name,"w")
    f.create_dataset('loss',data=loss)
    f.create_dataset('val_loss',data=valoss)
    f.close()

In [ ]:
#import matplotlib.pyplot as plt
import matplotlib.cm as cmx
import matplotlib.colors as colors

def get_cmap(N):
    
    '''Returns a function that maps each index in 0, 1, ... N-1 to a distinct 
    RGB color.'''
    color_norm  = colors.Normalize(vmin=0, vmax=N-1)
    scalar_map = cmx.ScalarMappable(norm=color_norm, cmap='hsv') 
    def map_index_to_rgb_color(index):
        return scalar_map.to_rgba(index)
    return map_index_to_rgb_color

In [ ]:
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
%matplotlib inline
from keras.layers.advanced_activations import PReLU, LeakyReLU

In [ ]:
def hist(y, n=25):
    plt.figure(figsize=(8,8))
    l = y.min()
    r = y.max()
    w = ((r-l)/float(y.shape[0]*1.))
    plt.hist(y, histtype='step', bins=n)
    plt.show()

#generates a Gaussian 
def gaussian(batch_size, mu=0.,sigma=0.5):
    sample = np.random.normal(mu, sigma, batch_size)
    sample.sort()
    return np.reshape(sample,(len(sample),1))

def noise(batch_size, m=-2.5, M=2.5):
    a = np.linspace(m, M, batch_size) + np.random.random(batch_size) * 0.01
    return np.reshape(a,(len(a),1))

def noise_g(batch_size, mu=0.,sigma=1.0):
    sample = np.random.normal(mu, sigma, batch_size)+np.random.random(batch_size) * 0.01
    sample.sort()
    return np.reshape(sample,(len(sample),1))

#y=gaussian(batch_size=10000, mu=5, sigma=5)
#print y.shape
#hist(y)

In [ ]:
plt.hist(noise_g(1000, -2 , 0.5))

In [ ]:
def plot_loss(losses,modelname="GAN"):
    #display.clear_output(wait=True)
    #display.display(plt.gcf())
    plt.figure(figsize=(16,9))
    plt.plot(losses["d_f"], label='discriminative loss on fake', alpha=0.8)
    plt.plot(losses["d_t"], label='discriminative loss on true', alpha=0.8)
    plt.plot(losses["d"], label = 'discriminative loss', alpha=0.8)
    plt.plot(losses["g"], label='generative loss', alpha=0.8)
    #plt.yscale('log')
    plt.xlabel('Epochs', fontsize=22)
    plt.ylabel('Loss', fontsize=22)
    plt.legend(fontsize=22, loc='best', fancybox=True, framealpha=0.)
    plt.savefig("%s_loss.png"%modelname)
    plt.show()


In [ ]:
opt = Adam(lr=1e-4)#, decay = 0.009)
dopt = Adam(lr=1e-3)#, decay = 0.0009)
from keras.models import Sequential,Model
from keras.layers import concatenate, merge
def gen_f():
    
    g_input= Input(shape=[1])
    mean_input = Input(shape=[1])
    g = merge([g_input, mean_input])
    g = Dense(800)(g)
    g = Activation('relu')(g)
    g = BatchNormalization()(g)
    #g = Dropout(0.2)(g)
    g = Dense(400)(g)
    g = Activation('relu')(g)
    #g = Dropout(0.2)(g)
    g = Dense(400)(g)
    g = Activation('relu')(g)
    g = Dense(400)(g)
    g = Activation('relu')(g)
    #g = Dropout(0.2)(g)
    g = Activation('relu')(g)
    g = Dense(1, activation='linear')(g)
    G = Model(inputs=[g_input, mean_input], outputs=[g] )
    G.compile(loss='binary_crossentropy', optimizer=opt)
    return G

#def disc_f():

#    d_input= Input(shape=[1])
#    d = Dense(400)(d_input)
#    d = Activation('tanh')(d)
#    d = BatchNormalization()(d)
#    d = Dense(400)(d)
#    d = Activation('tanh')(d)
    #d = Dense(400)(d)
    #d = Activation('tanh')(d)
    #d = Flatten()(d)
#    d = MinibatchDiscrimination(nb_kernels=10, kernel_dim=1)(d)#, input_shape=(None, 400),)(d)
    #minibatch_featurizer = Lambda(minibatch_discriminator,
                                     # output_shape=minibatch_output_shape)
    #features = [d]
    
    #nb_features = 10
    #vspace_dim = 2

    # creates the kernel space for the minibatch discrimination
    #K_x = Dense3D(nb_features, vspace_dim)(x)
    #features.append(Activation('tanh')(minibatch_featurizer(K_x)))
    
    
    
#    d = Dense(1, kernel_initializer='normal', activation='sigmoid')(d)
#    D = Model(d_input,d)
#    D.compile(loss='binary_crossentropy', optimizer=dopt)
#    return D


In [ ]:
##From CaloGAN paper
from ops_CaloGAN import *
def minibatch_discriminator(x):
    """ Computes minibatch discrimination features from input tensor x"""
    diffs = K.expand_dims(x, 3) - K.expand_dims(K.permute_dimensions(x, [1, 2, 0]), 0)
    l1_norm = K.sum(K.abs(diffs), axis=2)
    return K.sum(K.exp(-l1_norm), axis=2)


def minibatch_output_shape(input_shape):
    """ Computes output shape for a minibatch discrimination layer"""
    shape = list(input_shape)
    assert len(shape) == 3  # only valid for 3D tensors
    return tuple(shape[:2])




def build_discriminator(d_input, mbd=False):
    """ Generator sub-component for the CaloGAN ---- reusing this recipe for now (KD)

    Args:
    -----
        mdb: bool, perform feature level minibatch discrimination
        
    Returns:
    --------
        a keras tensor of features

    """
    
    d = Dense(800)(d_input)
    d = Activation('tanh')(d)
    d = Dropout(0.2)(d)
    d = Dense(800)(d)
    d = Activation('tanh')(d)
    d = Dropout(0.2)(d)
    d = Dense(400)(d)
    d = Activation('tanh')(d)
    d = Dropout(0.2)(d)
    d = Dense(400)(d)
    d = Activation('tanh')(d)
    d = Dropout(0.2)(d)
    #d = Dense(400)(d)
    #d = Activation('tanh')(d)
    #d = Dropout(0.2)(d)
    #d = Dense(400)(d)
    #d = Activation('tanh')(d)
    #d = Dropout(0.2)(d)
    #d = BatchNormalization()(d)
    if mbd:
        minibatch_featurizer = Lambda(minibatch_discriminator,
                                      output_shape=minibatch_output_shape)

        features = [d]
        nb_features = 30
        vspace_dim = 2

        # creates the kernel space for the minibatch discrimination
        K_d = Dense3D(nb_features, vspace_dim)(d)
        features.append(Activation('tanh')(minibatch_featurizer(K_d)))
        features = concatenate(features)
        #p = concatenate([features])
        D = Dense(1, kernel_initializer='normal', activation='sigmoid')(features)
        discriminator = Model(d_input,D)
        discriminator.compile(loss='binary_crossentropy', optimizer=dopt)
        return discriminator
    
    else:
        d = Dense(400)(d)
        d = Activation('tanh')(d)
        d = Dense(1, kernel_initializer='normal', activation='sigmoid')(d)
        discriminator = Model(d_input, d)
        discriminator.compile(loss='binary_crossentropy', optimizer=dopt)
        return discriminator

In [ ]:
from keras.models import Sequential,Model
from keras.layers import add, concatenate, multiply
mbd=True
def GAN_f(generator, discriminator):

    noise=Input(shape=[1])
    mean = Input(shape=[1])
    g=generator(([noise, mean]))
    d=discriminator(g)
    discriminator.trainable = False
    GAN = Model([noise,mean], d)
    GAN.compile(loss='binary_crossentropy', optimizer=opt)
    #GAN.summary()
    return GAN


D = build_discriminator(d_input=Input(shape=[1]), mbd=True)
D.summary()
G = gen_f()
G.summary()
Model = GAN_f(G, D)
D.trainable = True
Model.summary()
#p_g = G.predict(noise(10000))
#plt.hist(p_g)
#plt.show()
#p_d = D.predict(np.ones(5000))
#p_d = np.concatenate((p_d, D.predict(np.zeros(5000))))
#plt.hist(p_d)

In [ ]:
batch_size = 2000
import random
losses = {"d_f":[], "d_t":[], "d":[], "g":[]}
def train(G, D, M, epoch, mean):
    
    true_disc_x = gaussian(batch_size,mu=mean,sigma=0.5)
    sd = np.std(true_disc_x)
    #mean = np.mean(true_disc_x)
    mean_col = mean*np.ones((batch_size,1))
    latent = noise_g(batch_size, mu=0., sigma=10.) #noise_g(batch_size, mu=0., sigma = 1.)  # m=true_disc_x.min(), M=true_disc_x.max())
    true_disc_y = np.zeros(batch_size)
    fake_disc_x = G.predict([latent, mean_col])
    #one-sided label smoothing: random.uniform(0.8,1.0)*
    fake_disc_y = np.ones(batch_size)
    
    x_disc = np.concatenate((true_disc_x, fake_disc_x))
    x_disc = x_disc.reshape((x_disc.shape[0], 1))
    y_disc = np.concatenate((true_disc_y, fake_disc_y))
    y_disc = y_disc.reshape((y_disc.shape[0], 1))
    
    G.trainable = False
    D.trainable = True
    d_loss_true = D.train_on_batch(true_disc_x, true_disc_y)
    d_loss_fake = D.train_on_batch(fake_disc_x, fake_disc_y)
    d_loss = (d_loss_true + d_loss_fake) / 2.0
    losses["d_f"].append(d_loss_fake)
    losses["d_t"].append(d_loss_true)
    losses["d"].append(d_loss)
    
    x_gen_1 = latent
    x_gen_2 = mean_col
    y_gen = np.zeros(batch_size)    
       
    G.trainable = True 
    D.trainable = False    
    
    GAN_loss = M.train_on_batch([x_gen_1, x_gen_2] , y_gen)
    losses["g"].append(GAN_loss)
    
    return d_loss, GAN_loss

In [ ]:
pretrain_disc = False
if pretrain_disc ==True:
    #temp (pretrain dicriminator)
    true_disc_x = gaussian(batch_size,mu=0.,sigma=0.5)
    mean_col = mean*np.ones((batch_size,1))
    latent = noise_g(batch_size, mu=0, sigma=10.)
    true_disc_y = np.zeros(batch_size)
    fake_disc_x = G.predict([latent, mean_col])
    fake_disc_y = np.ones(batch_size)
    

    #x_disc = np.concatenate((true_disc_x, fake_disc_x))
    #x_disc = x_disc.reshape((x_disc.shape[0], 1))
    #y_disc = np.concatenate((true_disc_y, fake_disc_y))

    G.trainable = False
    D.trainable = True
    #d_loss_true = D.train_on_batch(true_disc_x, true_disc_y)
    #d_loss_fake = D.train_on_batch(fake_disc_x, fake_disc_y)
    d_loss = D.train_on_batch(true_disc_x, true_disc_y)
    d_loss = D.train_on_batch(fake_disc_x, fake_disc_y)

    print d_loss


nb_epochs = 5001
mean_arr = [-1., 0., 1., 1., 0., -1.]

for i in range(0,nb_epochs):
    mc=0
    for m_true in mean_arr:
        d_loss, g_loss = train(G,D,Model,i,m_true)
        #losses["d"].append(d_loss)
        #losses["g"].append(g_loss)  

        if i % 50 == 0:
            print("Epoch {0}, Discriminator loss:{1}, GAN loss:{2}".format(i + 1, d_loss, g_loss))
            #overlay GAN output on the Gaussian we want

            true_disc = gaussian(10000,mu=m_true,sigma=0.5)
            sd = np.std(true_disc)
            mean = np.mean(true_disc)
            print "For true: mean and s.d. = ", mean, sd
            mean_col = m_true*np.ones((10000,1))
            latent = noise_g(10000,mu=0.,sigma=10.)

            G.trainable = False
            D.trainable = False
            fake_disc = G.predict([latent, mean_col])

            sd = np.std(fake_disc)
            mean = np.mean(fake_disc)
            print "For generator: mean and s.d. = ", mean, sd

            plt.figure(figsize=(16,9))
            plt.title("True (mu=0, sigma=0.5) vs. Generator Output", fontsize=24)
            plt.hist(latent, color='r', histtype='step', linewidth=5, bins = 50, range = (-10,10), label = "Noise", alpha=0.5)
            plt.hist(true_disc, color='g', histtype='step', linewidth=5, bins = 50, range = (-10,10), label = "True", alpha=0.5)
            plt.hist(fake_disc, color='b', histtype='step', linewidth=5, bins = 50, range = (-10,10), label = 'Fake/Generator', alpha=0.5)
            plt.legend(loc='best', fontsize=22, fancybox=True, framealpha=0.)
            # plt.ylim(0,0.6)
            plt.rc('xtick', labelsize = 22)
            plt.rc('ytick', labelsize = 22)
            plt.show()     
            #print losses

            true_pred = D.predict(true_disc)
            fake_pred = D.predict(fake_disc)

            plt.figure(figsize=(16,9))
            plt.title("Discriminator prediction", fontsize=24)
            plt.hist(true_pred, color='r', histtype='step', linewidth=5, bins = 25, range = (-2.,2.), label = "w/ true input", alpha=0.5)
            plt.hist(fake_pred, color='b', histtype='step', linewidth=5, bins = 25, range = (-2.,2.), label = 'w/ fake input', alpha=0.5)
            plt.legend(loc='best', fontsize=22, fancybox=True, framealpha=0.)
            plt.xlim(0.,1.)
            plt.rc('xtick', labelsize = 22)
            plt.rc('ytick', labelsize = 22)
            plt.show()


            plot_loss(losses)
    #savemodel(model=G, name="Gen_mbd_varg_%d"%mc)
    #mc=mc+1

In [ ]:
p = G.predict(noise(batch_size=100000, m=-3., M=3.))
plt.hist(p)

In [ ]:
sd = np.std(p, dtype=np.float64)
v = np.var(p, dtype=np.float64)
mean = np.mean(p, dtype=np.float64)
from keras.layers import Activation, Dense
print mean, sd
savemodel(model=G, name="Gen_mbd_varg")
#save_model(model=D, filepath="/home/kaustuv1993/Desktop/H2bb/Models_and_Weights/Disc_mbd_1")

#savemodel(model=Model, name="GaussiGAN_mbd_1")

In [ ]:
        ###################
        batch_size = 1000000    
        true_disc = gaussian(batch_size,mu=2.,sigma=0.5)
        true_disc.sort()
        print true_disc.min(), true_disc.max()
        sd = np.std(true_disc)
        mean = 2
        mean_col = mean*np.ones((batch_size,1))
        latent = noise_g(batch_size,mu=0.,sigma=10.)
        G.trainable = False
        D.trainable = False
        fake_disc = G.predict([latent, mean_col])
        latent.sort()
        G.trainable = False
        D.trainable = False
        fake_disc = G.predict([latent,mean_col])
        #print fake_disc
        sd = np.std(fake_disc)
        mean = np.mean(fake_disc)
        print "For generator: mean and s.d = ", mean, sd
        ###################
        
        plt.figure(figsize=(16,9))
        plt.title("Original (mu=0, sigma=0.5) vs. Generator Output", fontsize=24)
        plt.hist(true_disc, color='r', histtype='step', linewidth=5, bins = 50, range = (-4.5,2.5), label = "Original", alpha=0.5, normed=True)
        plt.hist(fake_disc, color='b', histtype='step', linewidth=5, bins = 50, range = (-4.5,2.5), label = 'Generator', alpha=0.5, normed=True)
        plt.legend(loc='best', fontsize=22, fancybox=True, framealpha=0.)
        # plt.ylim(0,0.6)
        plt.rc('xtick', labelsize = 22)
        plt.rc('ytick', labelsize = 22)
        #plt.savefig("GaussiGAN_mbd_unilat_gen_PDF.png")
        plt.show()     
        #print losses
        
        true_pred = D.predict(true_disc)
        fake_pred = D.predict(fake_disc)
            
        plt.figure(figsize=(16,9))
        plt.title("Discriminator prediction", fontsize=24)
        plt.hist(true_pred, color='r', histtype='step', linewidth=5, bins = 10, range = (0.,1.), label = "w/ true input", alpha=0.5, normed=True)
        plt.hist(fake_pred, color='b', histtype='step', linewidth=5, bins = 10, range = (0.,1.), label = 'w/ fake input', alpha=0.5, normed=True)
        plt.legend(loc='best', fontsize=22, fancybox=True, framealpha=0.)
        plt.xlim(0.1,0.9)
        plt.rc('xtick', labelsize = 22)
        plt.rc('ytick', labelsize = 22)
        #plt.savefig("GaussiGAN_mbd_unilat_disc_pred.png")
        plt.show()

In [ ]:
batch_size=1000000
import scipy.stats as stats        

true = gaussian(batch_size=batch_size, mu=0., sigma = 0.5)
true.sort()
tmean = np.mean(true)
tstd = np.std(true)
tpdf = stats.norm.pdf(true, tmean, tstd)
tcdf = stats.norm.cdf(true, tmean, tstd)
#x = np.linspace(true.min(), true.max(), 100000)

latent = noise_g(batch_size=batch_size, mu=0., sigma = 1.0)
latent.sort()
G_x = G.predict(latent)

H_G,X_G = np.histogram(G_x, bins = 100, normed = True )
dx = X_G[1] - X_G[0]
F_G[1:] = np.cumsum(H_G)*dx




plt.figure(figsize=(16,9))
plt.title("CDF/PDF overlay", fontsize=24)

plt.plot(true, tpdf, 
         color='b', linestyle='--', linewidth=5, label = r"$\mathrm{True\ PDF\ (\mu=0,\ \sigma=0.5)}$")

plt.plot(true, tcdf, 
         color = 'k', linestyle='--', linewidth=5, label = "True CDF")

plt.hist(G_x, histtype='step', bins =100, normed=True,
         color = 'r', linewidth=3, label ="Generator PDF")

plt.plot(X_G, F_G, 
         color = 'y', linewidth=3, label="Generator CDF")

plt.legend(loc='best', fontsize=22, fancybox=True, framealpha=0.)
plt.rc('xtick', labelsize = 22)
plt.rc('ytick', labelsize = 22)
plt.savefig("GaussiGAN_1_mbd_g1_overlay.png")
plt.show()


In [ ]:
print F_G.shape, X_G.shape

In [ ]:
        from scipy.stats import norm
        batch_size=100000
        
        
        g_inp = noise_g(batch_size=100000, mu=0., sigma = 1.0)
        x = np.linspace(g_inp.min(),g_inp.max(), 100000)
        x.sort()
        g_inp.sort()
        dx = ((x.max()-x.min())/100000.)
        G_x = G.predict(g_inp)
        
        
        plt.figure(figsize=(16,9))
        plt.title("CDF/PDF overlay", fontsize=24)
        plt.hist(G_x, color='chartreuse', bins=50,  linewidth=3,label = "Generator predictions", normed = True, alpha=0.5)
        #G_x /= (dx*G_x).sum()
        #CDF_gen = np.cumsum(G_x*dx)
        plt.plot(g_inp, G_x, color = 'burlywood', linewidth=3, label ="Generator CDF")
        plt.plot(x, norm.pdf(x, loc = 0., scale = 0.5), color='c', linewidth=3, label = "True PDF")
        plt.plot(x, norm.cdf(x, loc = 0., scale = 0.5), color = 'm', linewidth=3, label="True CDF")
        plt.plot(x, norm.ppf(x, loc = 0., scale = 0.5), color = 'y', linewidth=3, label="True PPF")
        plt.plot(g_inp, norm.pdf(G_x, loc = 0., scale = 0.5), color = 'r', linestyle = '--', linewidth=3, label="Gen. PDF")
        plt.plot(g_inp, norm.cdf(G_x, loc = 0., scale = 0.5), color = 'g', linestyle = '--', linewidth=3, label="Gen. CDF")
        plt.plot(g_inp, norm.ppf(G_x, loc = 0., scale = 0.5), color = 'b', linestyle = '--', linewidth=3, label="Gen. PPF")
        plt.legend(loc='best', fontsize=22, fancybox=True, framealpha=0.)
        #plt.ylim(-2,2)
        plt.rc('xtick', labelsize = 22)
        plt.rc('ytick', labelsize = 22)
        plt.savefig("GaussiGAN_1_mbd_g1_overlay.png")
        plt.show()
        

In [ ]:
2+2

In [ ]:
from keras.utils import plot_model
plot_model(G, show_shapes=True, to_file='VarGaussiGen.png')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

RANGE = 5


# p_d(x)
class DataDistribution(object):
    def __init__(self, mu, sigma):
        self.mu = mu
        self.sigma = sigma

    def sample(self, N):
        samples = np.random.normal(self.mu, self.sigma, N)
        samples.sort()
        return samples


# p_z(z)
class NoiseDistribution(object):
    def __init__(self, range):
        self.range = range

    # equally spaced samples + noise
    def sample(self, N):
        offset = np.random.random(N) * (float(self.range) / N)
        samples = np.linspace(-self.range, self.range, N) + offset
        return samples


# G(z)
def generator(x, n_hidden=32):

    # initializers
    w_init = tf.truncated_normal_initializer(stddev=2)
    b_init = tf.constant_initializer(0.)

    # 1st hidden layer
    w0 = tf.get_variable('w0', [x.get_shape()[1], n_hidden], initializer=w_init)
    b0 = tf.get_variable('b0', [n_hidden], initializer=b_init)
    h0 = tf.nn.relu(tf.matmul(x, w0) + b0)

    # output layer
    w1 = tf.get_variable('w1', [h0.get_shape()[1], 1], initializer=w_init)
    b1 = tf.get_variable('b1', [1], initializer=b_init)
    o = tf.matmul(h0, w1) + b1

    return o


# D(x)
def discriminator(x, n_hidden=32):

    # initializers
    w_init = tf.contrib.layers.variance_scaling_initializer()
    b_init = tf.constant_initializer(0.)

    # 1st hidden layer
    w0 = tf.get_variable('w0', [x.get_shape()[1], n_hidden], initializer=w_init)
    b0 = tf.get_variable('b0', [n_hidden], initializer=b_init)
    h0 = tf.nn.relu(tf.matmul(x, w0) + b0)

    # output layer
    w1 = tf.get_variable('w1', [h0.get_shape()[1], 1], initializer=w_init)
    b1 = tf.get_variable('b1', [1], initializer=b_init)
    o = tf.sigmoid(tf.matmul(h0, w1) + b1)

    return o


# re-used for optimizing all networks
def optimizer(loss, var_list, num_decay_steps=400, initial_learning_rate=0.03):
    decay = 0.95
    batch = tf.Variable(0)
    learning_rate = tf.train.exponential_decay(
        initial_learning_rate,
        batch,
        num_decay_steps,
        decay,
        staircase=True
    )
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(
        loss,
        global_step=batch,
        var_list=var_list
    )
    return optimizer


# plot decision boundaries (init, pre-trained, trained) and p_data, p_g
class ResultPlot(object):
    def __init__(self, num_points, num_bins, mu, sigma):
        self.num_points = num_points    # number of data points to be evaluated
        self.num_bins = num_bins        # number of bins to get histogram

        self.mu = mu                    # mu of p_data
        self.sigma = sigma              # sigma of p_data

        self.xs = np.linspace(-RANGE, RANGE, num_points)    # positions of data
        self.bins = np.linspace(-RANGE, RANGE, num_bins)    # positions of bins

    def show_results(self, db_init, db_pre_trained, db_trained, pd, pg, save_img=True):
        db_x = np.linspace(-RANGE, RANGE, len(db_trained))
        p_x = np.linspace(-RANGE, RANGE, len(pd))
        f, ax = plt.subplots(1)
        ax.plot(db_x, db_init, 'g--', linewidth=2, label='db_init')
        ax.plot(db_x, db_pre_trained, 'c--', linewidth=2, label='db_pre_trained')
        ax.plot(db_x, db_trained, 'g-', linewidth=2, label='db_trained')
        ax.set_ylim(0, max(1, np.max(pd) * 1.1))
        ax.set_xlim(max(self.mu - self.sigma * 3, -RANGE * 0.9), min(self.mu + self.sigma * 3, RANGE * 0.9))
        plt.plot(p_x, pd, 'b-', linewidth=2, label='real data')
        plt.plot(p_x, pg, 'r-', linewidth=2, label='generated data')
        plt.title('1D Generative Adversarial Network: ' + '(mu : %3g,' % self.mu + ' sigma : %3g)' % self.sigma)
        plt.xlabel('Data values')
        plt.ylabel('Probability density')
        plt.legend()
        plt.grid(True)

        if save_img:
            plt.savefig('GAN_1d_gaussian' + '_mu_%g' % self.mu + '_sigma_%g' % self.sigma + '.png')

        plt.show()


def main():
    """ parameters """
    # p_data
    mu = 0
    sigma = 1

    # training
    B = 150                 # batch-size
    LR = 0.03               # learning rate (generator uses half of this)
    TRAIN_ITERS = 3000      # number of iterations

    # network
    n_hidden = 32           # number of hidden layers

    """ build graph """

    # networks : pre-trained discriminator
    with tf.variable_scope('D_pre'):
        pre_input = tf.placeholder(tf.float32, shape=(None, 1))
        pre_labels = tf.placeholder(tf.float32, shape=(None, 1))
        D_pre = discriminator(pre_input, n_hidden)

    # networks : generator
    with tf.variable_scope('Gen'):
        z = tf.placeholder(tf.float32, shape=(None, 1))
        G_z = generator(z, n_hidden)

    # networks : discriminator
    with tf.variable_scope('Disc') as scope:
        x = tf.placeholder(tf.float32, shape=(None, 1))
        D_real = discriminator(x, n_hidden)
        scope.reuse_variables()
        D_fake = discriminator(G_z, n_hidden)

    # loss for each network
    eps = 1e-2  # to prevent log(0) case
    pre_loss = tf.reduce_mean(tf.square(D_pre - pre_labels))
    loss_g = tf.reduce_mean(-tf.log(D_fake + eps))
    loss_d = tf.reduce_mean(-tf.log(D_real + eps) - tf.log(1 - D_fake + eps))

    # trainable variables for each network
    d_pre_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='D_pre')
    d_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='Disc')
    g_params = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='Gen')

    # optimizer for each network
    pre_opt = optimizer(pre_loss, d_pre_params, 400, LR)
    opt_d = optimizer(loss_d, d_params, 400, LR)
    opt_g = optimizer(loss_g, g_params, 400, LR / 2)

    """ training """

    # open session and initialize all variables
    sess = tf.InteractiveSession()
    tf.global_variables_initializer().run()

    # sources
    p_data = DataDistribution(mu, sigma)
    p_z = NoiseDistribution(RANGE)

    # class for result-plot
    plot = ResultPlot(10000, 20, mu, sigma)

    # plot : initial decision boundary
    db_init = np.zeros((plot.num_points, 1))
    for i in range(plot.num_points // B):
        db_init[B * i:B * (i + 1)] = sess.run(D_real, {x: np.reshape(plot.xs[B * i:B * (i + 1)], (B, 1))})

    # pre-training discriminator
    num_pretrain_steps = 1000
    for step in range(num_pretrain_steps):

        print('pre-training :  %d/%d' % (step + 1, num_pretrain_steps))

        # Object of pre-training is to make decision boundary as similar as pdf of data (i.e. p_data)
        # Since p_data is unknown in real situation, we get histogram and estimate pdf
        N = 1000
        d = p_data.sample(N)
        n_bins = 100
        histc, edges = np.histogram(d, bins=n_bins, density=True)

        # Estimated pdf is used as labels after normalization
        max_histc = np.max(histc)
        min_histc = np.min(histc)
        labels = (histc - min_histc) / (max_histc - min_histc)
        d = edges[1:]

        # Execute one training step
        sess.run([pre_loss, pre_opt],
                 {pre_input: np.reshape(d, (n_bins, 1)), pre_labels: np.reshape(labels, (n_bins, 1))})
    print('pre-training finished!')

    # store pre-trained variables
    weightsD = sess.run(d_pre_params)

    # copy weights from pre-training over to new D network
    tf.global_variables_initializer().run()
    for i, v in enumerate(d_params):
        sess.run(v.assign(weightsD[i]))

    # plot : pre-trained decision boundary
    db_pre_trained = np.zeros((plot.num_points, 1))
    for i in range(plot.num_points // B):
        db_pre_trained[B * i:B * (i + 1)] = sess.run(D_real, {x: np.reshape(plot.xs[B * i:B * (i + 1)], (B, 1))})

    # training-loop
    for step in range(TRAIN_ITERS):

        np.random.seed(np.random.randint(0, TRAIN_ITERS))

        # update discriminator
        x_ = p_data.sample(B)
        z_ = p_z.sample(B)

        loss_d_, _ = sess.run([loss_d, opt_d], {x: np.reshape(x_, (B, 1)), z: np.reshape(z_, (B, 1))})

        # update generator
        z_ = p_z.sample(B)
        loss_g_, _ = sess.run([loss_g, opt_g], {z: np.reshape(z_, (B, 1))})

        if step % 10 == 0:
            print('[%d/%d]: loss_d : %.3f, loss_g : %.3f' % (step, TRAIN_ITERS, loss_d_, loss_g_))

    """ show results """

    # plot : trained decision boundary
    db_trained = np.zeros((plot.num_points, 1))
    for i in range(plot.num_points // B):
        db_trained[B * i:B * (i + 1)] = sess.run(D_real, {x: np.reshape(plot.xs[B * i:B * (i + 1)], (B, 1))})

    # plot : pdf of data distribution
    d = p_data.sample(plot.num_points)
    pd, _ = np.histogram(d, bins=plot.bins, density=True)

    # plot : pdf of generated samples
    zs = np.linspace(-RANGE, RANGE, plot.num_points)
    g = np.zeros((plot.num_points, 1))
    for i in range(plot.num_points // B):
        g[B * i:B * (i + 1)] = sess.run(G_z, {z: np.reshape(zs[B * i:B * (i + 1)], (B, 1))})
    pg, _ = np.histogram(g, bins=plot.bins, density=True)

    # plot results
    plot.show_results(db_init, db_pre_trained, db_trained, pd, pg, save_img=True)

    sess.close()


if __name__ == '__main__':
    main()